# Data Preparation

In [1]:
import pandas as pd
from numpy import nan
import numpy as np
import scipy.stats as stats
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
violations = pd.read_csv(
    'data/AllViolations.csv', 
    parse_dates = ['date'], 
    infer_datetime_format = True,
    index_col = 0
)

In [3]:
import json
from pandas.io.json import json_normalize

with open('data/yelp_academic_dataset_business.json', 'r') as f:
    data = f.readlines()
data = map(lambda x: x.rstrip(), data)
data_json_str = '[' + (',').join(data) + ']'

business = pd.read_json(data_json_str)
business = business[[
    'name',
    'full_address',
    'city',
    'business_id',
    'review_count', 
    'stars',
    'latitude',
    'longitude'
]]
business.columns = [
    'name',     
    'address',
    'city',
    'yelp_id',
    'reviews',
    'rating',
    'latitude',
    'longitude'
]

%reset_selective -f data_json_str
%reset_selective -f data

In [4]:
idtable = pd.read_csv(
    'data/restaurant_ids_to_yelp_ids.csv'
)
idtable = pd.concat([
        idtable[['restaurant_id','yelp_id_0']].rename(columns = {'yelp_id_0':'yelp_id'}),
        idtable[idtable.yelp_id_1.notnull()][['restaurant_id','yelp_id_1']].rename(columns = {'yelp_id_1':'yelp_id'}),
        idtable[idtable.yelp_id_2.notnull()][['restaurant_id','yelp_id_2']].rename(columns = {'yelp_id_2':'yelp_id'}),
        idtable[idtable.yelp_id_3.notnull()][['restaurant_id','yelp_id_3']].rename(columns = {'yelp_id_3':'yelp_id'})
], ignore_index = True)

In [5]:
violations = violations.merge(
    idtable, 
    on = 'restaurant_id'
).merge(
    business, 
    on = 'yelp_id'
).drop(['yelp_id', 'address', 'city'], 1)
violations = violations[violations.date >= '2011-08-01']

In [6]:
services = pd.read_csv(
    'data/311__Service_Requests.csv', 
    usecols = [
       'OPEN_DT', 'CLOSED_DT', 'CASE_STATUS', 'CLOSURE_REASON', 'CASE_TITLE', 'SUBJECT', 'REASON',
       'TYPE', 'LATITUDE', 'LONGITUDE'
    ],
    parse_dates=['OPEN_DT', 'CLOSED_DT'], 
    infer_datetime_format = True
)
services.columns = [
    'open',         # date the complaint was registered
    'closed',       # date the complaint was resolved
    'status',       # open or closed
    'closure',      # details of how the complaint was closed (NaN if open)
    'title',        # description of the issue
    'subject',      # category of the issue (e.g. public works, civil rights, animal control, etc.)
    'reason',       # more specific category (often very similar to the "title")
    'type',         # usually identical to the "title" 
    'latitude',      
    'longitude'      
]
services = services.sort_values(['open', 'reason'])

In [7]:
reason = services.reason.value_counts().head(45).reset_index()['index']

In [8]:
business.to_pickle('data/business.pkl')
idtable.to_pickle('data/idtable.pkl')
services.to_pickle('data/services.pkl')
reason.to_pickle('data/reason.pkl')

In [9]:
pd.options.mode.chained_assignment = None
for j in range(45):
    violations[reason[j]] = nan

violations['delay'] = nan
    
for row in range(len(violations)):
    c = (violations.latitude.iloc[row], violations.longitude.iloc[row])
    ri = violations.restaurant_id.iloc[row]
    t = violations.date.iloc[row]
    tlast = violations[(violations.date <= t - timedelta(days = 1)) & (violations.restaurant_id == ri)].date.max()
    violations['delay'].iloc[row] = t - tlast
    s = services[
            (services.open < t - timedelta(days = 5)) &
            (services.open > t - timedelta(days = 15)) &
            ((services.latitude - c[0])**2 + (5/7*(services.longitude - c[1]))**2 < 0.0025)
    ]
    violations.iloc[row,10:55] = s.groupby('reason').count().iloc[:,0][reason].fillna(0).values
        
violations['delay'] = (violations.delay) / (timedelta(days=1))
violations['delay'] = violations.delay.astype(float)

In [10]:
violations.to_pickle('data/violations.pkl')